# Weather Regulation Prediction System - Quick Start Tutorial

This notebook provides a comprehensive introduction to the Weather Regulation Prediction System. You'll learn how to:

1. Load and configure the system
2. Prepare weather and regulation data
3. Train different machine learning models
4. Evaluate and compare model performance
5. Generate reports and visualizations

## Prerequisites

Make sure you have installed all required dependencies:

```bash
pip install -r requirements.txt
```

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. System Configuration

The system uses a configuration-based approach for managing experiments. Let's start by creating a basic configuration.

In [ ]:
from config import ExperimentConfig, DataConfig, TrainingConfig, RandomForestConfig, LSTMConfig
from config_parser import ConfigParser

# Create a basic configuration
config = ExperimentConfig(
    name="quick_start_experiment",
    description="Introduction to the weather regulation prediction system",
    data=DataConfig(
        airports=["EGLL"],  # London Heathrow
        start_date="2023-01-01",
        end_date="2023-01-31",
        time_resolution="30min"
    ),
    training=TrainingConfig(
        test_size=0.2,
        validation_size=0.2,
        random_state=42,
        cross_validation=True,
        cv_folds=5
    ),
    models={
        'random_forest': RandomForestConfig(
            n_estimators=100,
            max_depth=10,
            random_state=42
        )
    }
)

print(f"Configuration created for experiment: {config.name}")
print(f"Target airport: {config.data.airports[0]}")
print(f"Models to train: {list(config.models.keys())}")

## 2. Generate Sample Data

For this tutorial, we'll generate synthetic weather and regulation data that resembles real aviation data.

In [ ]:
# Generate synthetic weather data
np.random.seed(42)
n_samples = 1000

# Create realistic weather patterns
timestamps = pd.date_range('2023-01-01', periods=n_samples, freq='30min')

# Temperature with daily cycle
hour_of_day = timestamps.hour
temp_base = 10 + 8 * np.sin(2 * np.pi * hour_of_day / 24)  # Daily temperature cycle
temperature = temp_base + np.random.normal(0, 3, n_samples)  # Add noise

# Pressure (realistic values)
pressure = np.random.normal(1013, 15, n_samples)

# Wind speed (typically lower at night)
wind_base = 5 + 3 * np.sin(2 * np.pi * hour_of_day / 24 + np.pi/2)
wind_speed = np.abs(wind_base + np.random.normal(0, 2, n_samples))

# Wind direction
wind_direction = np.random.uniform(0, 360, n_samples)

# Visibility (lower during poor weather)
visibility = np.random.lognormal(9.2, 0.5, n_samples)  # Log-normal distribution
visibility = np.clip(visibility, 1000, 20000)  # Realistic range

# Humidity
humidity = np.random.beta(3, 2, n_samples) * 100  # Beta distribution

# Weather codes (simplified)
weather_codes = np.random.choice(['CLR', 'FEW', 'SCT', 'BKN', 'OVC', 'RA', 'SN', 'FG'], 
                                n_samples, p=[0.3, 0.2, 0.15, 0.15, 0.1, 0.05, 0.02, 0.03])

# Create weather DataFrame
weather_data = pd.DataFrame({
    'timestamp': timestamps,
    'airport': 'EGLL',
    'temperature': temperature,
    'pressure': pressure,
    'wind_speed': wind_speed,
    'wind_direction': wind_direction,
    'visibility': visibility,
    'humidity': humidity,
    'weather_code': weather_codes
})

print(f"Generated {len(weather_data)} weather observations")
print("\nWeather data sample:")
print(weather_data.head())

In [ ]:
# Generate regulation data
# Regulations are more likely during poor weather conditions

# Create weather severity score
weather_severity = (
    (weather_data['visibility'] < 5000).astype(int) * 0.3 +  # Low visibility
    (weather_data['wind_speed'] > 15).astype(int) * 0.2 +    # High wind
    weather_data['weather_code'].isin(['RA', 'SN', 'FG']).astype(int) * 0.3 +  # Precipitation/fog
    (weather_data['temperature'] < 0).astype(int) * 0.2      # Freezing conditions
)

# Probability of regulation based on weather severity
regulation_prob = 0.1 + 0.4 * weather_severity  # Base 10% + up to 40% for severe weather

# Generate regulations with some randomness
has_regulation = np.random.binomial(1, regulation_prob)

# Create regulation DataFrame
regulation_data = pd.DataFrame({
    'timestamp': timestamps,
    'airport': 'EGLL',
    'has_regulation': has_regulation,
    'regulation_type': np.where(has_regulation, 
                               np.random.choice(['WX', 'ATC', 'EQ'], len(has_regulation)), 
                               'None'),
    'severity_score': weather_severity
})

regulation_rate = has_regulation.mean()
print(f"Generated {len(regulation_data)} regulation records")
print(f"Regulation rate: {regulation_rate:.1%}")
print(f"Total regulations: {has_regulation.sum()}")

print("\nRegulation data sample:")
print(regulation_data.head())

## 3. Data Exploration and Visualization

Let's explore our generated data to understand the patterns.

In [ ]:
# Create visualizations of the data
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Weather Data Exploration', fontsize=16)

# Temperature over time
axes[0, 0].plot(weather_data['timestamp'], weather_data['temperature'])
axes[0, 0].set_title('Temperature over Time')
axes[0, 0].set_ylabel('Temperature (°C)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Pressure distribution
axes[0, 1].hist(weather_data['pressure'], bins=30, alpha=0.7)
axes[0, 1].set_title('Pressure Distribution')
axes[0, 1].set_xlabel('Pressure (hPa)')
axes[0, 1].set_ylabel('Frequency')

# Wind speed vs regulations
reg_data = weather_data.merge(regulation_data[['timestamp', 'has_regulation']], on='timestamp')
for reg_status, label in [(0, 'No Regulation'), (1, 'Regulation')]:
    data_subset = reg_data[reg_data['has_regulation'] == reg_status]
    axes[0, 2].scatter(data_subset['wind_speed'], data_subset['visibility'], 
                      alpha=0.6, label=label, s=20)
axes[0, 2].set_title('Wind Speed vs Visibility (by Regulation Status)')
axes[0, 2].set_xlabel('Wind Speed (kt)')
axes[0, 2].set_ylabel('Visibility (m)')
axes[0, 2].legend()

# Regulations by hour of day
hourly_regs = regulation_data.copy()
hourly_regs['hour'] = hourly_regs['timestamp'].dt.hour
hourly_reg_rate = hourly_regs.groupby('hour')['has_regulation'].mean()
axes[1, 0].bar(hourly_reg_rate.index, hourly_reg_rate.values)
axes[1, 0].set_title('Regulation Rate by Hour of Day')
axes[1, 0].set_xlabel('Hour')
axes[1, 0].set_ylabel('Regulation Rate')

# Weather code distribution
weather_counts = weather_data['weather_code'].value_counts()
axes[1, 1].bar(weather_counts.index, weather_counts.values)
axes[1, 1].set_title('Weather Code Distribution')
axes[1, 1].set_xlabel('Weather Code')
axes[1, 1].set_ylabel('Count')
axes[1, 1].tick_params(axis='x', rotation=45)

# Correlation heatmap
numeric_cols = ['temperature', 'pressure', 'wind_speed', 'visibility', 'humidity']
corr_data = weather_data[numeric_cols].corr()
sns.heatmap(corr_data, annot=True, cmap='coolwarm', center=0, ax=axes[1, 2])
axes[1, 2].set_title('Weather Variable Correlations')

plt.tight_layout()
plt.show()

# Summary statistics
print("Weather Data Summary:")
print(weather_data[numeric_cols].describe())

## 4. Data Processing and Feature Engineering

Now let's use the system's data pipeline to process and enhance our data.

In [ ]:
from data.feature_engineering import WeatherFeatureEngineer, TimeSeriesFeatureEngineer
from data.preprocessing import PreprocessingPipeline, TimeSeriesScaler

# Create weather-specific features
weather_engineer = WeatherFeatureEngineer()
enhanced_weather = weather_engineer.create_features(weather_data)

print(f"Original features: {weather_data.shape[1]}")
print(f"Enhanced features: {enhanced_weather.shape[1]}")
print(f"New features added: {enhanced_weather.shape[1] - weather_data.shape[1]}")

# Show new features
new_features = [col for col in enhanced_weather.columns if col not in weather_data.columns]
print(f"\nNew features: {new_features}")

# Display sample of enhanced data
print("\nEnhanced weather data sample:")
print(enhanced_weather[['timestamp', 'temperature', 'flight_category', 'weather_severity', 'wind_components_u', 'wind_components_v']].head())

In [ ]:
# Create time series features
ts_engineer = TimeSeriesFeatureEngineer()
enhanced_data = ts_engineer.create_features(
    enhanced_weather,
    timestamp_col='timestamp',
    value_cols=['temperature', 'pressure', 'wind_speed'],
    lags=[1, 3, 6],  # 30min, 1.5h, 3h lags
    rolling_windows=[6, 12]  # 3h, 6h rolling windows
)

print(f"After time series engineering: {enhanced_data.shape[1]} features")

# Show some time series features
ts_features = [col for col in enhanced_data.columns if 'lag_' in col or 'rolling_' in col or col in ['hour', 'day_of_week', 'month']]
print(f"\nTime series features: {ts_features[:10]}...")  # Show first 10

In [ ]:
# Combine with regulation data to create final dataset
final_data = enhanced_data.merge(
    regulation_data[['timestamp', 'has_regulation']], 
    on='timestamp', 
    how='inner'
)

# Remove rows with NaN (due to lag features)
final_data = final_data.dropna()

print(f"Final dataset shape: {final_data.shape}")
print(f"Regulation rate in final dataset: {final_data['has_regulation'].mean():.1%}")

# Prepare features and target
feature_cols = [col for col in final_data.columns if col not in ['timestamp', 'airport', 'has_regulation', 'weather_code']]
X = final_data[feature_cols]
y = final_data['has_regulation']

print(f"\nFeature matrix shape: {X.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")

## 5. Model Training

Let's train different models using the system's training pipeline.

In [ ]:
from sklearn.model_selection import train_test_split
from models.random_forest import RandomForestModel
from models.fnn import FNNModel
from training.trainer import Trainer
from config import FNNConfig

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Initialize trainer
trainer = Trainer()

# Train Random Forest
print("\n=== Training Random Forest ===")
rf_config = RandomForestConfig(n_estimators=100, max_depth=10, random_state=42)
rf_model = RandomForestModel(rf_config)

rf_results = trainer.train_model(
    model=rf_model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    model_name="random_forest"
)

print(f"Random Forest Results:")
print(f"- Accuracy: {rf_results['accuracy']:.3f}")
print(f"- Precision: {rf_results['precision']:.3f}")
print(f"- Recall: {rf_results['recall']:.3f}")
print(f"- F1 Score: {rf_results['f1_score']:.3f}")
print(f"- Training Time: {rf_results['training_time']:.2f}s")

In [ ]:
# Train Feedforward Neural Network
print("\n=== Training Feedforward Neural Network ===")
fnn_config = FNNConfig(
    hidden_layer_sizes=[100, 50],
    max_iter=200,
    random_state=42,
    early_stopping=True
)
fnn_model = FNNModel(fnn_config)

fnn_results = trainer.train_model(
    model=fnn_model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    model_name="feedforward_nn"
)

print(f"Feedforward NN Results:")
print(f"- Accuracy: {fnn_results['accuracy']:.3f}")
print(f"- Precision: {fnn_results['precision']:.3f}")
print(f"- Recall: {fnn_results['recall']:.3f}")
print(f"- F1 Score: {fnn_results['f1_score']:.3f}")
print(f"- Training Time: {fnn_results['training_time']:.2f}s")

## 6. Model Evaluation and Comparison

Let's evaluate our models on the test set and compare their performance.

In [ ]:
# Evaluate models on test set
print("=== Test Set Evaluation ===")

# Random Forest test evaluation
rf_test_metrics = rf_model.evaluate(X_test, y_test)
rf_predictions = rf_model.predict(X_test)
rf_probabilities = rf_model.predict_proba(X_test)

print(f"\nRandom Forest Test Results:")
print(f"- Accuracy: {rf_test_metrics.accuracy:.3f}")
print(f"- Precision: {rf_test_metrics.precision:.3f}")
print(f"- Recall: {rf_test_metrics.recall:.3f}")
print(f"- F1 Score: {rf_test_metrics.f1_score:.3f}")
print(f"- AUC-ROC: {rf_test_metrics.auc_roc:.3f}")

# FNN test evaluation
fnn_test_metrics = fnn_model.evaluate(X_test, y_test)
fnn_predictions = fnn_model.predict(X_test)
fnn_probabilities = fnn_model.predict_proba(X_test)

print(f"\nFeedforward NN Test Results:")
print(f"- Accuracy: {fnn_test_metrics.accuracy:.3f}")
print(f"- Precision: {fnn_test_metrics.precision:.3f}")
print(f"- Recall: {fnn_test_metrics.recall:.3f}")
print(f"- F1 Score: {fnn_test_metrics.f1_score:.3f}")
print(f"- AUC-ROC: {fnn_test_metrics.auc_roc:.3f}")

In [ ]:
# Create comparison visualizations
from visualization.plots import ModelVisualizer
from sklearn.metrics import confusion_matrix

visualizer = ModelVisualizer()

# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Confusion Matrices Comparison', fontsize=16)

# Random Forest confusion matrix
rf_cm = confusion_matrix(y_test, rf_predictions)
sns.heatmap(rf_cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Random Forest')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# FNN confusion matrix
fnn_cm = confusion_matrix(y_test, fnn_predictions)
sns.heatmap(fnn_cm, annot=True, fmt='d', cmap='Blues', ax=axes[1])
axes[1].set_title('Feedforward Neural Network')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

plt.tight_layout()
plt.show()

In [ ]:
# ROC curves comparison
from sklearn.metrics import roc_curve, auc

plt.figure(figsize=(10, 8))

# Random Forest ROC
rf_fpr, rf_tpr, _ = roc_curve(y_test, rf_probabilities[:, 1])
rf_auc = auc(rf_fpr, rf_tpr)
plt.plot(rf_fpr, rf_tpr, linewidth=2, label=f'Random Forest (AUC = {rf_auc:.3f})')

# FNN ROC
fnn_fpr, fnn_tpr, _ = roc_curve(y_test, fnn_probabilities[:, 1])
fnn_auc = auc(fnn_fpr, fnn_tpr)
plt.plot(fnn_fpr, fnn_tpr, linewidth=2, label=f'Feedforward NN (AUC = {fnn_auc:.3f})')

# Random baseline
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random (AUC = 0.500)')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves Comparison')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Feature importance analysis (Random Forest)
rf_importance = rf_model.get_feature_importance()

if rf_importance is not None:
    # Plot top 15 most important features
    top_features = rf_importance.head(15)
    
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title('Top 15 Most Important Features (Random Forest)')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("Top 10 Most Important Features:")
    print(rf_importance.head(10).to_string(index=False))
else:
    print("Feature importance not available for this model.")

## 7. Hyperparameter Tuning

Let's demonstrate hyperparameter tuning to improve model performance.

In [ ]:
from training.hyperparameter_tuning import GridSearchTuner

# Define parameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10]
}

print("Starting hyperparameter tuning...")
print(f"Parameter grid: {param_grid}")
print(f"Total combinations: {np.prod([len(v) for v in param_grid.values()])}")

# Create new model for tuning
tuning_model = RandomForestModel(RandomForestConfig(random_state=42))

# Perform grid search
tuner = GridSearchTuner(scoring='f1', n_jobs=-1)
tuning_result = tuner.tune(
    model=tuning_model,
    param_grid=param_grid,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    cv=3  # 3-fold CV for speed
)

print(f"\nBest parameters: {tuning_result.best_params}")
print(f"Best CV score: {tuning_result.best_score:.3f}")
print(f"Number of trials: {len(tuning_result.all_results)}")

In [ ]:
# Train model with best parameters
print("\n=== Training Optimized Random Forest ===")
optimized_config = RandomForestConfig(**tuning_result.best_params, random_state=42)
optimized_model = RandomForestModel(optimized_config)

optimized_results = trainer.train_model(
    model=optimized_model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    model_name="optimized_random_forest"
)

# Test the optimized model
optimized_test_metrics = optimized_model.evaluate(X_test, y_test)

print(f"\nOptimized Random Forest Test Results:")
print(f"- Accuracy: {optimized_test_metrics.accuracy:.3f}")
print(f"- Precision: {optimized_test_metrics.precision:.3f}")
print(f"- Recall: {optimized_test_metrics.recall:.3f}")
print(f"- F1 Score: {optimized_test_metrics.f1_score:.3f}")
print(f"- AUC-ROC: {optimized_test_metrics.auc_roc:.3f}")

# Compare with original
print(f"\nImprovement over original:")
print(f"- Accuracy: {optimized_test_metrics.accuracy - rf_test_metrics.accuracy:+.3f}")
print(f"- F1 Score: {optimized_test_metrics.f1_score - rf_test_metrics.f1_score:+.3f}")
print(f"- AUC-ROC: {optimized_test_metrics.auc_roc - rf_test_metrics.auc_roc:+.3f}")

## 8. Results Management and Reporting

Let's save our results and generate a comprehensive report.

In [ ]:
from results.results_manager import ResultsManager, ExperimentResult, ModelResult
from datetime import datetime

# Create results manager
results_manager = ResultsManager(base_path="./tutorial_results")

# Create experiment result
experiment_result = ExperimentResult(
    experiment_id="tutorial_experiment_001",
    experiment_name="Quick Start Tutorial Experiment",
    timestamp=datetime.now(),
    config=config
)

# Create model results for each trained model
models_data = [
    ("Random Forest", rf_test_metrics, rf_results['training_time']),
    ("Feedforward NN", fnn_test_metrics, fnn_results['training_time']),
    ("Optimized RF", optimized_test_metrics, optimized_results['training_time'])
]

for model_name, metrics, training_time in models_data:
    model_result = ModelResult(
        model_name=model_name,
        model_type=model_name.split()[0],  # First word as type
        timestamp=datetime.now(),
        config={},
        training_time=training_time,
        test_accuracy=metrics.accuracy,
        test_precision=metrics.precision,
        test_recall=metrics.recall,
        test_f1=metrics.f1_score,
        test_auc=metrics.auc_roc,
        confusion_matrix=metrics.confusion_matrix
    )
    experiment_result.add_model_result(model_result)

# Save experiment
experiment_id = results_manager.save_experiment_result(experiment_result)
print(f"Experiment saved with ID: {experiment_id}")

# Display experiment summary
print(f"\nExperiment Summary:")
print(f"- Best Model: {experiment_result.best_model}")
print(f"- Best Accuracy: {experiment_result.best_accuracy:.3f}")
print(f"- Models Trained: {len(experiment_result.model_results)}")
print(f"- Total Training Time: {experiment_result.total_training_time:.2f}s")

In [ ]:
# Generate comprehensive report
from results.report_generator import ReportGenerator

# Create report generator
report_generator = ReportGenerator()

# Generate HTML report
try:
    html_report_path = report_generator.generate_report(
        experiment=experiment_result,
        format='html',
        output_path='tutorial_experiment_report.html',
        include_visualizations=True
    )
    print(f"HTML report generated: {html_report_path}")
except Exception as e:
    print(f"Could not generate HTML report: {e}")

# Generate Markdown report
try:
    md_report_path = report_generator.generate_report(
        experiment=experiment_result,
        format='markdown',
        output_path='tutorial_experiment_report.md'
    )
    print(f"Markdown report generated: {md_report_path}")
except Exception as e:
    print(f"Could not generate Markdown report: {e}")

## 9. Model Comparison Summary

Let's create a final comparison of all our models.

In [ ]:
# Create comparison DataFrame
comparison_data = []

for model_name, model_result in experiment_result.model_results.items():
    comparison_data.append({
        'Model': model_name,
        'Accuracy': model_result.test_accuracy,
        'Precision': model_result.test_precision,
        'Recall': model_result.test_recall,
        'F1 Score': model_result.test_f1,
        'AUC-ROC': model_result.test_auc,
        'Training Time (s)': model_result.training_time
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.round(3)

print("Model Comparison Summary:")
print("=" * 80)
print(comparison_df.to_string(index=False))

# Find best model for each metric
print("\nBest Models by Metric:")
print("=" * 40)
for metric in ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC-ROC']:
    best_idx = comparison_df[metric].idxmax()
    best_model = comparison_df.loc[best_idx, 'Model']
    best_value = comparison_df.loc[best_idx, metric]
    print(f"{metric:>12}: {best_model} ({best_value:.3f})")

In [ ]:
# Final visualization: Model performance radar chart
import math

# Prepare data for radar chart
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC-ROC']
angles = [n / float(len(metrics)) * 2 * math.pi for n in range(len(metrics))]
angles += angles[:1]  # Complete the circle

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

colors = ['blue', 'red', 'green']
for i, (_, row) in enumerate(comparison_df.iterrows()):
    values = [row[metric] for metric in metrics]
    values += values[:1]  # Complete the circle
    
    ax.plot(angles, values, 'o-', linewidth=2, label=row['Model'], color=colors[i])
    ax.fill(angles, values, alpha=0.25, color=colors[i])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics)
ax.set_ylim(0, 1)
ax.set_title('Model Performance Comparison (Radar Chart)', size=16, y=1.1)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax.grid(True)

plt.tight_layout()
plt.show()

## 10. Conclusion and Next Steps

Congratulations! You've successfully completed the quick start tutorial for the Weather Regulation Prediction System. Here's what you've learned:

### What You've Accomplished:

1. **Data Generation**: Created realistic synthetic weather and regulation data
2. **Feature Engineering**: Enhanced raw data with domain-specific and time series features
3. **Model Training**: Trained multiple machine learning models (Random Forest, Neural Network)
4. **Hyperparameter Tuning**: Optimized model performance using grid search
5. **Evaluation**: Compared models using various metrics and visualizations
6. **Results Management**: Saved and organized experimental results
7. **Reporting**: Generated comprehensive reports in multiple formats

### Key Insights from This Tutorial:

- Weather features significantly impact regulation prediction accuracy
- Time series features (lags, rolling statistics) improve model performance
- Hyperparameter tuning can provide meaningful improvements
- Different models excel at different aspects (precision vs recall trade-offs)

### Next Steps:

1. **Try More Models**: Experiment with LSTM, CNN, Transformer, or Ensemble models
2. **Advanced Tuning**: Use Bayesian optimization for more efficient hyperparameter search
3. **Real Data**: Apply the system to actual METAR and ATFM regulation data
4. **Feature Engineering**: Explore additional weather-specific features
5. **Cross-Validation**: Implement time series cross-validation for more robust evaluation
6. **Production Deployment**: Set up model monitoring and retraining pipelines

### Additional Resources:

- Check out the other tutorial notebooks for advanced topics
- Refer to the API documentation for detailed function references
- Explore the configuration system for customizing experiments
- Use the interactive dashboard for model comparison and analysis

Thank you for using the Weather Regulation Prediction System!